In [1]:
import pandas as pd
import numpy as np
import re

Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. Получившийся скрипт необходимо выложить в гит репозиторий и прислать ссылку в качестве результата прохождения тестового задания. Данные выкладывать в гит не следует. 

Главные задачи, которые должен выполнять скрипт:
1. Извлекать реплики с приветствием – где менеджер поздоровался. 
2. Извлекать реплики, где менеджер представил себя. 
3. Извлекать имя менеджера. 
4. Извлекать название компании. 
5. Извлекать реплики, где менеджер попрощался.
6. Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

**Рекомендации:**
- Сделать локальную копию файла test_data.csv, в исходнике никакие данные не менять!
- Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.
- Для выполнения задачи можно использовать любые библиотеки и NLP модели. 
- Попробуйте учесть возможные синонимичные выражения, которые могут помочь с извлечением данных сущностей. 

Результат выполнения задачи отправить на почту hr@bewise.ai, также указать свое ФИО и ссылку на резюме.

Работы принимаются до 09.09.2022, далее в течении 7 календарных дней мы анализируем результаты и приглашаем на собеседования подходящих кандидатов.


In [2]:
# загружаем вспомогательную информацию с мужскими и женскими именами

female_names = open("ru_female_names.txt", encoding = 'utf-8').read().lower().split('\n')
male_names = open("ru_male_names.txt", encoding = 'utf-8').read().lower().split('\n')

In [5]:
# считываем данные
df = pd.read_csv('test_data.csv')

# создаем новую колонку для записи результатов
df['insight'] = np.nan

# подсчет количества диалогов
dialogues = df['dlg_id'].unique()

In [6]:
# для каждого диалога в датасете
for i in dialogues:
    
#     метки - соблюдались ли правила для приветствия и прощания
    hi = False
    bye = False
    print('Диалог', i)
#     выбираем только строки соответствующего диалога, которые произносил менеджер
    df_ = df[(df['dlg_id'] == i)&(df['role'] == 'manager')]
    
#     для каждой строки в выбранной совокупности
    for j in range(0, len(df_)):
        
#         извлекаем текст
        text = df_['text'].iloc[j]
#     шаблон для приветствия
        rs1 = re.findall(r"(?i)здравст*|добрый*\s\w+|привет*", text)
        if len(rs1) != 0:
            print('Приветствие:', text)
            df['insight'].iloc[df_[df_['text']==df_['text'].iloc[j]].index] = 'Greeting'
            hi = True
            
#         шаблон для представления
        rs2 = re.findall(r"(?i)меня\s|зовут*\s\w+", text)
        if len(rs2) != 0:
            print('Менеджер представил себя:', text)
            df['insight'].iloc[df_[df_['text']==df_['text'].iloc[j]].index] = 'Introduction'
            
#             поиск имен
            words = text.split(' ')
            for word in words:
                if word in female_names or word in male_names:
                    print('Имя менеджера:', word)
#             шаблон для названия компании
            rsc = re.findall(r"компания\s\w+\sбизнес|компания\s\w+", text)
            print('Название компании:', rsc[0][9:])
        
#         шаблон для прощания
        rs3 = re.findall(r"(?i)всего\sхор*|до*\sсвид*", text)
        if len(rs3) != 0:
            print('Прощание:', text)
            df['insight'].iloc[df_[df_['text']==df_['text'].iloc[j]].index] = 'Saying_bye'
            bye = True
#     проверка соблюдения требований
    print('Проверка требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»: ')            
    if hi and bye:
        print('Выполнено')
    elif hi and bye==False:
        print('Не выполнено - Прощание')
    elif hi==False and bye:
        print('Не выполнено - Приветствие')
    else:
        print('Не выполнено')     
    print()
    print('-----------')
    print()

Диалог 0
Приветствие: Алло здравствуйте
Менеджер представил себя: Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
Имя менеджера: ангелина
Название компании: диджитал бизнес
Прощание: Всего хорошего до свидания
Проверка требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»: 
Выполнено

-----------

Диалог 1
Приветствие: Алло здравствуйте
Менеджер представил себя: Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается
Имя менеджера: ангелина
Название компании: диджитал бизнес
Прощание: Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего
Прощание: До свидания
Проверка требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»: 
Выполнено

-----------

Диалог 2
Приветствие: Алло здравствуйте
Менеджер пред

In [7]:
df

,dlg_id,line_n,role,text,insight
0,0,0,client,Алло,NaN
1,0,1,manager,Алло здравствуйте,Greeting
2,0,2,client,Добрый день,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,Introduction
4,0,4,client,Ага,NaN
...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,NaN
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,NaN
477,5,140,client,Спасибо спасибо,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,NaN


In [8]:
df[df['insight']=='Greeting']

,dlg_id,line_n,role,text,insight
1,0,1,manager,Алло здравствуйте,Greeting
110,1,1,manager,Алло здравствуйте,Greeting
166,2,2,manager,Алло здравствуйте,Greeting
250,3,1,manager,Алло дмитрий добрый день,Greeting


In [9]:
df[df['insight']=='Introduction']

,dlg_id,line_n,role,text,insight
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,Introduction
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,Introduction
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,Introduction
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,Introduction


In [10]:
df[df['insight']=='Saying_bye']

,dlg_id,line_n,role,text,insight
108,0,108,manager,Всего хорошего до свидания,Saying_bye
162,1,53,manager,Угу да вижу я эту почту хорошо тогда исправлю ...,Saying_bye
163,1,54,manager,До свидания,Saying_bye
335,4,33,manager,Во вторник все ну с вами да тогда до вторника ...,Saying_bye
479,5,142,manager,Ну до свидания хорошего вечера,Saying_bye


In [11]:
df.to_csv('test_result.csv')